In [2]:
import numpy as np
import pandas as pd

np.random.seed(42)
# [각주 1] seed(시드)는 "난수(랜덤)"의 시작점을 고정하는 장치입니다.
# 같은 코드를 다시 실행해도 같은 데이터가 생성되도록 해서 '재현 가능성'을 확보합니다.
# 포트폴리오/업무에서 재현 가능성은 신뢰도와 직결됩니다.

n = 3000
# [각주 2] 작업 로그의 '행(row) 개수' = 작업(task) 개수입니다.
# 너무 작으면 패턴이 불안정하고, 너무 크면 설명이 과해집니다.
# 1~5천 정도는 포트폴리오 실험/EDA에 딱 적당한 크기입니다.

df = pd.DataFrame({
    "task_id": range(1, n+1),
    # [각주 3] task_id는 작업 단위 식별자입니다.
    # 실제 운영 로그에서도 task_id/order_id 같은 키가 존재합니다.
    
    "operator_type": np.random.choice(["new", "experienced"], n, p=[0.4, 0.6]),
    # [각주 4] operator_type은 "신규/숙련"을 뜻합니다.
    # p=[0.4, 0.6]은 신규 40%, 숙련 60% 비율로 구성한다는 의미입니다.
    # 이 비율은 회사마다 다를 수 있지만, 신규가 일정 비율 존재한다는 현실을 반영합니다.
    
    "category": np.random.choice(["electronics", "fashion", "living"], n),
    # [각주 5] category는 오류가 특정 카테고리에서 더 잘 발생하는지 분석하기 위한 축입니다.
    # (이 코드는 아직 category별 오류확률을 다르게 주지는 않았지만,
    #  다음 단계에서 "electronics에서 오류율이 높다" 같은 시나리오도 쉽게 추가 가능합니다.)
    
    "period": np.random.choice(["before", "after"], n, p=[0.5, 0.5]),
    # [각주 6] period는 SOP/교육 개선 "전(before) / 후(after)"를 뜻합니다.
    # 개선 전후 비교를 하려면 반드시 이런 구분 컬럼이 필요합니다.
})



In [3]:
# 1) 베이스라인 오류 생성 (당신이 만든 세계관 유지)
def generate_error(row):
    # [각주 7] 이 함수는 "오류가 발생할 확률"을 운영 가설에 맞게 부여합니다.
    # 여기서 중요한 건 "운영 논리"를 반영했다는 점입니다.
    # 단순 랜덤이 아니라, 조건에 따라 오류 확률이 달라집니다.
    
    if row["operator_type"] == "new" and row["period"] == "before":
        return np.random.binomial(1, 0.25)
        # [각주 8] binomial(1, p)는 0/1(실패/성공)을 확률 p로 생성합니다.
        # 여기서는 "오류 발생"을 1로 두었기 때문에,
        # p=0.25는 '오류가 25% 확률로 발생'한다는 뜻입니다.
        # 가설: 신규 + 개선 전(before)은 오류율이 높다.
        
    elif row["operator_type"] == "new" and row["period"] == "after":
        return np.random.binomial(1, 0.15)
        # [각주 9] 신규라도 SOP/교육이 개선된 after에서는 오류율이 낮아진다고 가정합니다.
        # 즉, 개선 전 25% → 개선 후 15%로 떨어지는 구조를 "베이스라인 세계관"으로 설계합니다.
        
    else:
        return np.random.binomial(1, 0.08)
        # [각주 10] 숙련자는 전반적으로 오류율이 낮다고 가정합니다(8%).
        # (experienced + before/after 모두 8%로 단순화)
        # 포트폴리오에서는 너무 복잡하게 시작할 필요가 없고,
        # 핵심 메시지(신규가 더 어렵다)를 전달하는 수준이 적당합니다.

df["error_flag"] = df.apply(generate_error, axis=1)
# [각주 11] apply는 각 행(row)에 대해 generate_error 함수를 실행합니다.
# axis=1은 "행 단위로 적용"이라는 뜻입니다.
# 결과는 error_flag 컬럼에 저장됩니다. (오류면 1, 아니면 0)



In [ ]:
# 2) 실험 대상만 추출: 신규 + after
exp = df[(df["operator_type"]=="new") & (df["period"]=="after")].copy()
# [각주 12] 여기서 매우 중요한 실험 설계 포인트가 나옵니다.
# A/B 테스트에서는 "같은 모집단"에서 control vs treatment를 비교해야 합니다.
# 그래서 모집단을 '신규 오퍼레이터 + after 기간'으로 고정합니다.
# 즉, 동일한 조건(신규 + after)에서 개선안 적용 여부만 다르게 만들어 비교합니다.
# copy()는 경고(SettingWithCopyWarning) 방지 및 독립 데이터프레임 확보 목적입니다.




In [ ]:
# 3) 동일 모집단에서 A/B 랜덤 배정
np.random.seed(123)
# [각주 13] 실험 배정 자체도 재현 가능해야 합니다.
# seed를 따로 두는 이유:
#   - 데이터 생성 seed(42)와
#   - 실험 배정 seed(123)를
# 분리해두면, 실험 배정만 바꾸거나 유지하는 통제가 쉬워집니다.

exp["group"] = np.random.choice(["control", "treatment"], size=len(exp), p=[0.5, 0.5])
# [각주 14] 동일 모집단(exp)에서 절반은 control, 절반은 treatment로 무작위 배정합니다.
# 이것이 A/B 테스트의 핵심인 "랜덤 할당(Randomization)"입니다.
# 랜덤 할당이 있어야 confound(혼입 변수) 위험이 줄어듭니다.



In [ ]:
# 4) 실험 결과 오류 생성: 같은 조건에서 group만 다르게
np.random.seed(999)
# [각주 15] 이제 "실험 결과"를 생성합니다.
# seed를 고정해서, 결과 또한 다시 실행해도 동일하게 나오도록 합니다.
# (실험 재현성)

exp["error_flag_exp"] = np.where(
    exp["group"]=="treatment",
    np.random.binomial(1, 0.12, size=len(exp)),   # 개선안 적용
    np.random.binomial(1, 0.15, size=len(exp))    # 기존 after 신규 수준(베이스라인)
)
# [각주 16] np.where(condition, A, B)는 조건이면 A, 아니면 B를 선택합니다.
# - treatment면 오류확률 12% (개선안 적용)
# - control이면 오류확률 15% (개선안 미적용: baseline after 수준)
#
# 여기서 포인트:
# 1) 모집단은 동일: 신규+after로 고정
# 2) 차이는 오직 group(개선안 적용 여부)뿐
# 3) 따라서 두 그룹의 error_rate 차이는 "개선안의 효과"로 해석 가능
#
# size=len(exp)는 exp 데이터 길이만큼 0/1 값을 생성하겠다는 뜻입니다.



In [ ]:
# 5) 결과 요약 (PAGE 8 근거)
result = exp.groupby("group")["error_flag_exp"].mean().reset_index()
# [각주 17] group별 평균(mean)은 0/1의 평균이므로 곧 '오류율(error rate)'입니다.
# 예: error_flag_exp 평균이 0.15면 오류율 15%
# reset_index()는 결과를 데이터프레임 형태로 보기 좋게 만들기 위해 사용합니다.

result
# [각주 18] 이 표가 PAGE 8 슬라이드의 "실험 결과" 근거가 됩니다.
